<a href="https://colab.research.google.com/github/hrnrxb/AI-News-Aggregator-Bot/blob/main/Telegram_AI_News_Aggregator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install python-telegram-bot feedparser beautifulsoup4 requests

In [78]:
from telegram import Bot
import os

# Your Telegram bot token.
# For local testing, you can place it directly here.
# For production (GitHub Actions), use GitHub Secrets for security.
TELEGRAM_BOT_TOKEN = ""

# Your Telegram channel ID where the bot will send messages.
# This can be the channel's username (e.g., "@YourChannelName")
# or the numeric ID for private channels (e.g., -100123456789).
TELEGRAM_CHANNEL_ID = ""

# Initialize the Telegram Bot object with your token.
bot = Bot(token=TELEGRAM_BOT_TOKEN)

print("Initial Telegram bot setup complete.")

Initial Telegram bot setup complete.


In [80]:
import asyncio

async def test_telegram_message():
    """
    Sends a test message to the configured Telegram channel.
    Verifies if the bot can successfully send messages.
    """
    try:
        await bot.send_message(chat_id=TELEGRAM_CHANNEL_ID, text="HelloWWWW! This is a test message from  news bot.")
        print("Test message successfully sent to Telegram.")
    except Exception as e:
        print(f"Error sending test message: {e}")
        print("Please ensure your bot token and channel ID are correct, and the bot is an admin in the channel.")

# Execute the asynchronous test function.
await test_telegram_message()

Test message successfully sent to Telegram.


In [81]:
import os

# File to store previously sent links.
# This will be replaced by a more persistent storage (like SQLite) for GitHub Actions.
SENT_LINKS_FILE = "sent_links.txt"

def load_sent_links():
    """Loads previously sent links from the file."""
    if not os.path.exists(SENT_LINKS_FILE):
        return set()
    with open(SENT_LINKS_FILE, "r", encoding="utf-8") as f:
        return set(f.read().splitlines())

def save_sent_link(link):
    """Appends a new link to the file of sent links."""
    with open(SENT_LINKS_FILE, "a", encoding="utf-8") as f:
        f.write(link + "\n")

# Load existing sent links when the script starts.
sent_links = load_sent_links()

print(f"Number of previously sent links loaded: {len(sent_links)}")

Number of previously sent links loaded: 70


In [82]:
import feedparser
from bs4 import BeautifulSoup # Make sure BeautifulSoup is imported at the top of your script

def get_rss_feed_items(feed_url, source_name, limit=10):
    """
    Retrieves new news items from an RSS feed, including title, link, and summary.
    No emoji prefix is added here, as it's handled by format_telegram_post.

    Args:
        feed_url (str): The URL of the RSS feed.
        source_name (str): The name of the source (e.g., "Hugging Face Blog").
        limit (int): Maximum number of items to return.

    Returns:
        list: A list of tuples (title, link, summary, source_name_for_emoji_lookup).
    """
    print(f"Retrieving news from RSS: {source_name} ({feed_url})")
    items = []
    try:
        feed = feedparser.parse(feed_url)
        for entry in feed.entries[:limit]:
            title = getattr(entry, 'title', "No Title")
            link = getattr(entry, 'link', None)
            # Try to get summary or description, otherwise use title as a fallback
            summary = getattr(entry, 'summary', getattr(entry, 'description', title))

            # Clean up HTML tags from summary if any
            summary = BeautifulSoup(summary, "html.parser").get_text(separator=' ', strip=True)
            summary = (summary[:200] + '...') if len(summary) > 200 else summary # Truncate summary

            if link and link not in sent_links:
                # Format title: "Source Name: Actual Title"
                # Pass source_name again as the fourth item for emoji lookup in format_telegram_post
                items.append((f"{source_name}: {title}", link, summary, source_name))
            else:
                if link:
                    print(f"Duplicate link from {source_name} ignored: {link}")
        return items
    except Exception as e:
        print(f"Error retrieving RSS from {source_name}: {e}")
        return []

# Example usage (you can test this in a separate cell if needed)
# hf_blog_news = get_rss_feed_items("https://huggingface.co/blog/rss.xml", "Hugging Face Blog")
# print(f"Hugging Face Blog News (with summary): {hf_blog_news}")

# arxiv_nlp_news = get_rss_feed_items("http://export.arxiv.org/rss/cs.CL", "arXiv NLP")
# print(f"arXiv NLP News (with summary): {arxiv_nlp_news}")

# openai_blog_news = get_rss_feed_items("https://openai.com/blog/rss.xml", "OpenAI Blog")
# print(f"OpenAI Blog News (with summary): {openai_blog_news}")

# google_ai_blog_news = get_rss_feed_items("https://ai.googleblog.com/feeds/posts/default", "Google AI Blog")
# print(f"Google AI Blog News (with summary): {google_ai_blog_news}")

# papers_with_code_blog_news = get_rss_feed_items("https://paperswithcode.com/rss", "Papers With Code")
# print(f"Papers With Code Blog News (with summary): {papers_with_code_blog_news}")

# deepmind_blog_news = get_rss_feed_items("https://deepmind.com/blog/feed/basic/", "DeepMind Blog")
# print(f"DeepMind Blog News (with summary): {deepmind_blog_news}")

# kaggle_blog_news = get_rss_feed_items("https://www.kaggle.com/blog/rss/", "Kaggle Blog")
# print(f"Kaggle Blog News (with summary): {kaggle_blog_news}")

In [83]:
import requests
from bs4 import BeautifulSoup # Make sure BeautifulSoup is imported at the top of your script

def get_hacker_news_items(limit=10):
    """
    Retrieves top news items from Hacker News API, including title, link, and a short summary.
    No emoji prefix is added here, as it's handled by format_telegram_post.

    Args:
        limit (int): Maximum number of items to return.

    Returns:
        list: A list of tuples (title, link, summary, source_name_for_emoji_lookup).
    """
    source_name = "Hacker News" # Explicitly define source name for emoji lookup
    print(f"Retrieving news from Hacker News")
    items = []
    try:
        # Get IDs of top stories
        top_stories_url = "https://hacker-news.firebaseio.com/v0/topstories.json"
        top_story_ids = requests.get(top_stories_url).json()

        # Fetch details for each story ID
        for story_id in top_story_ids[:limit]:
            item_url = f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json"
            story_data = requests.get(item_url).json()

            title = story_data.get('title', "No Title")
            # If 'url' is not available, use the Hacker News discussion page link
            link = story_data.get('url', f"https://news.ycombinator.com/item?id={story_id}")

            # Hacker News API often lacks a direct summary; use 'text' or a default message
            summary = story_data.get('text', "Click to read more or join the discussion.")
            # Clean up HTML tags from summary and truncate it
            summary = BeautifulSoup(summary, "html.parser").get_text(separator=' ', strip=True)
            summary = (summary[:200] + '...') if len(summary) > 200 else summary

            if title and link and link not in sent_links:
                # Format title: "Source Name: Actual Title"
                # Pass source_name again as the fourth item for emoji lookup in format_telegram_post
                items.append((f"{source_name}: {title}", link, summary, source_name))
            else:
                if link:
                    print(f"Duplicate link from Hacker News ignored: {link}")
        return items
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving Hacker News: {e}")
        return []
    except Exception as e:
        print(f"General error in Hacker News: {e}")
        return []

# Example usage (you can test this in a separate cell if needed)
# hn_news = get_hacker_news_items(limit=5)
# print(f"Hacker News News (with summary): {hn_news}")

In [84]:
import requests
from bs4 import BeautifulSoup

def get_github_trending_repos(language="python", limit=10):
    """
    Scrapes trending GitHub repositories, including title, link, and description.
    No emoji prefix is added here, as it's handled by format_telegram_post.

    Args:
        language (str): The programming language to search for (e.g., "python").
        limit (int): Maximum number of repositories to return.

    Returns:
        list: A list of tuples (title, link, summary, source_name_for_emoji_lookup).
    """
    source_name = f"GitHub Trending ({language})" # Explicitly define source name for emoji lookup
    url = f"https://github.com/trending/{language}"
    print(f"Retrieving news from GitHub Trending: {language} ({url})")
    items = []
    try:
        res = requests.get(url)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        # Selector for trending repositories. Adapt if GitHub UI changes.
        for article_tag in soup.select("article.Box-row")[:limit]:
            title_tag = article_tag.select_one("h2 a")
            description_tag = article_tag.select_one("p.col-9") # Selector for description

            repo_name = title_tag.text.strip().replace("\n", "").replace(" ", "") if title_tag else "No Repository Name"
            link = "https://github.com" + title_tag["href"] if title_tag and "href" in title_tag.attrs else None
            summary = description_tag.text.strip() if description_tag else "No description available."
            summary = (summary[:200] + '...') if len(summary) > 200 else summary # Truncate summary

            if link and link not in sent_links:
                # Format title: "Source Name: Actual Title"
                # Pass source_name again as the fourth item for emoji lookup in format_telegram_post
                items.append((f"{source_name}: {repo_name}", link, summary, source_name))
            else:
                if link:
                    print(f"Duplicate link from GitHub Trending ignored: {link}")
        return items
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving GitHub Trending: {e}")
        return []
    except Exception as e:
        print(f"General error in GitHub Trending: {e}")
        return []

# Example usage (you can test this in a separate cell if needed)
# github_news = get_github_trending_repos(language="python")
# print(f"GitHub Trending News (with description): {github_news}")

In [85]:
def get_source_emoji(source_name):
    """
    Returns an appropriate emoji based on the news source's name.

    Args:
        source_name (str): The clean name of the news source (e.g., "Hugging Face Blog").

    Returns:
        str: An emoji string.
    """
    emoji_map = {
        # RSS Feeds
        "Hugging Face Blog": "🤗",  # Blog posts from Hugging Face
        "Hugging Face Paper": "📝", # Papers/research related to Hugging Face from JamesG's blog
        "ML Reddit": "🤖",          # Machine Learning discussions from Reddit
        "OpenAI Blog": "✨",        # OpenAI's latest breakthroughs
        "The Gradient": "📜",       # In-depth articles and essays on AI
        "Jay Alammar": "💡",        # Visual explanations and deep dives
        "DeepMind Blog": "🔬",       # Scientific research from DeepMind
        "AI From MIT News": "🎓",   # AI news directly from MIT
        "General News From MIT News": "🏛️", # Broader tech/science from MIT
        "Microsoft AI Blog": "💻",  # Microsoft's AI advancements
        "machinelearningmastery Blog": "👨‍🏫", # Practical ML tutorials
        "Nvidia AI Blog": "🚀",     # NVIDIA's AI hardware & software news
        "Towards Data Science": "📊",# Data Science articles and tutorials
        "Hacker News": "HN",        # No emoji, just "HN" prefix for classic feel

        # GitHub Trending (using specific language/topic emojis where applicable)
        "GitHub Trending (python)": "🐍",
        "GitHub Trending (jupyter-notebook)": "📓",
        "GitHub Trending (google colab)": "☁️", # Google Colab specific
        "GitHub Trending (Artificial Intelligence)": "🧠",
        "GitHub Trending (AI)": "🧠",
        "GitHub Trending (machine-learning)": "📈",
        "GitHub Trending (deep-learning)": "🌌", # Deep learning, cosmic feel
        "GitHub Trending (nlp)": "🗣️", # Natural Language Processing, talking
        "GitHub Trending (Natural Language Processing)": "🗣️",
        "GitHub Trending (CV)": "👁️", # Computer Vision, eye
        "GitHub Trending (Computer Vision)": "👁️",
        "GitHub Trending (Data Science)": "🧪", # Data Science, lab/experiment
        "GitHub Trending (Awesome Lists)": "⭐", # Curated lists
    }
    # Return the specific emoji for the source, or a default sparkle emoji if not found.
    return emoji_map.get(source_name, "✨")

print("Source emoji mapping defined.")

Source emoji mapping defined.


In [86]:
def collect_all_news():
    """
    Collects and aggregates news items from various sources (RSS, GitHub Trending, Hacker News).
    Ensures uniqueness of news items based on their links.

    Returns:
        list: A list of tuples, each containing (title, link, summary, source_name_for_emoji_lookup).
    """
    all_news_items = []

    print("\n--- Starting News Collection ---")

    # 1. Collect from RSS Feeds

    all_news_items.extend(get_rss_feed_items("https://hf.co/blog/feed.xml", "Hugging Face Blog", limit=10))
    all_news_items.extend(get_rss_feed_items("https://jamesg.blog/hf-papers.xml", "Hugging Face Paper", limit=10))
    all_news_items.extend(get_rss_feed_items("https://www.reddit.com/r/MachineLearning/.rss", "ML Reddit", limit=10))
    all_news_items.extend(get_rss_feed_items("https://openai.com/blog/rss.xml", "OpenAI Blog", limit=10))
    all_news_items.extend(get_rss_feed_items("https://thegradient.pub/rss/", "The Gradient", limit=10))
    all_news_items.extend(get_rss_feed_items("https://jalammar.github.io/feed.xml", "Jay Alammar", limit=10))
    all_news_items.extend(get_rss_feed_items("https://deepmind.google/blog/rss.xml", "DeepMind Blog", limit=10))
    all_news_items.extend(get_rss_feed_items("https://news.mit.edu/rss/topic/artificial-intelligence2", "AI From MIT News", limit=10))
    all_news_items.extend(get_rss_feed_items("https://www.technologyreview.com/feed/", "General News From MIT News", limit=10))
    all_news_items.extend(get_rss_feed_items("https://blogs.microsoft.com/ai/feed/", "Microsoft AI Blog", limit=10))
    all_news_items.extend(get_rss_feed_items("https://machinelearningmastery.com/blog/feed/", "machinelearningmastery Blog", limit=10))
    all_news_items.extend(get_rss_feed_items("https://blogs.nvidia.com/blog/category/ai/feed/", "Nvidia AI Blog", limit=10))
    all_news_items.extend(get_rss_feed_items("https://towardsdatascience.com/feed/", "Towards Data Science", limit=10))

    # 2. Collect from GitHub Trending

    all_news_items.extend(get_github_trending_repos(language="python", limit=10))
    all_news_items.extend(get_github_trending_repos(language="jupyter-notebook", limit=10))
    all_news_items.extend(get_github_trending_repos(language="google colab", limit=10))
    all_news_items.extend(get_github_trending_repos(language="Artificial Intelligence", limit=10))
    all_news_items.extend(get_github_trending_repos(language="AI", limit=10))
    all_news_items.extend(get_github_trending_repos(language="machine-learning", limit=10))
    all_news_items.extend(get_github_trending_repos(language="deep-learning", limit=10))
    all_news_items.extend(get_github_trending_repos(language="nlp", limit=10))
    all_news_items.extend(get_github_trending_repos(language="Natural Language Processing", limit=10))
    all_news_items.extend(get_github_trending_repos(language="CV", limit=10))
    all_news_items.extend(get_github_trending_repos(language="Computer Vision", limit=10))
    all_news_items.extend(get_github_trending_repos(language="Data Science", limit=10))
    all_news_items.extend(get_github_trending_repos(language="Awesome Lists", limit=10))


    # 3. Collect from Hacker News API
    all_news_items.extend(get_hacker_news_items(limit=10))

    # Remove duplicates. Using a dictionary preserves insertion order (Python 3.7+)
    # and ensures uniqueness based on the link.
    unique_news_items = {}
    # Unpack all 4 values from the tuple, as get_ functions now return 4 items.
    for title, link, summary, source_name_for_emoji_lookup in all_news_items:
        unique_news_items[link] = (title, link, summary, source_name_for_emoji_lookup)

    final_news_list = list(unique_news_items.values())

    print(f"Total unique new items collected: {len(final_news_list)}")
    print("--- News Collection Finished ---")

    # You can add sorting logic here if you want to sort items by date, etc.
    return final_news_list

# Example usage (you can run this in a separate cell for testing):
# fetched_news = collect_all_news()
# print("\nNew items ready for dispatch:")
# for title, link, summary, source_name_for_emoji_lookup in fetched_news:
#     print(f"- {source_name_for_emoji_lookup}: {title}\n  Summary: {summary}\n  Link: {link}\n")

In [87]:
from telegram.constants import ParseMode

# Define the channel ID for the footer.
# This global variable should be defined once at the top of your script (e.g., in Cell 2).
TELEGRAM_CHANNEL_ID_FOR_FOOTER = ""

def format_telegram_post(title, link, summary, source_name_for_emoji_lookup):
    """
    Formats the news item into an HTML string suitable for Telegram.
    Includes a source-specific emoji, title, summary, read more link, and channel ID.

    Args:
        title (str): The title of the news item.
        link (str): The URL of the news item.
        summary (str): A brief summary or description of the news item.
        source_name_for_emoji_lookup (str): The clean source name used to retrieve the specific emoji.

    Returns:
        str: An HTML formatted string for the Telegram post.
    """
    # Get the appropriate emoji based on the source name.
    # Make sure get_source_emoji function is defined and accessible.
    source_emoji = get_source_emoji(source_name_for_emoji_lookup)

    emoji_link = "🔗"
    emoji_channel = "📣"

    # Determine if a meaningful summary is available to include.
    if not summary or summary.strip().lower() in [
        "no description available.",
        "click to read more or join the discussion."
    ]:
        summary_text = ""  # If summary is generic or empty, don't include it.
    else:
        summary_text = f"\n\n{summary}"  # Add summary if available and meaningful.

    post_content = (
        f"{source_emoji} <b>{title}</b>"  # Source-specific emoji followed by bold title.
        f"{summary_text}"  # Optional summary text.
        f"\n\n{emoji_link} <a href='{link}'>Read More</a>"  # Hyperlinked "Read More".
        f"\n\n{emoji_channel} Channel: {TELEGRAM_CHANNEL_ID_FOR_FOOTER}"  # Channel ID in footer.
    )
    return post_content

# Example usage (you can test this in a separate cell if needed):
# formatted_message = format_telegram_post(
#     "DeepMind Blog: New Breakthrough in AI Research",
#     "https://example.com/ai-breakthrough",
#     "Researchers have developed a novel algorithm that significantly improves the efficiency of neural networks for complex tasks.",
#     "DeepMind Blog" # Pass the exact source name string here for emoji lookup
# )
# print(formatted_message)

# formatted_message_github = format_telegram_post(
#     "GitHub Trending (python): Awesome-Python-Project",
#     "https://github.com/Awesome-Python-Project",
#     "A curated list of awesome Python frameworks, libraries, software and resources.",
#     "GitHub Trending (python)" # Pass the exact source name string here
# )
# print(formatted_message_github)

In [94]:
import asyncio
from telegram.error import RetryAfter, TimedOut
from telegram.constants import ParseMode

async def send_news_to_telegram(news_items):
    """
    Sends collected news items to the Telegram channel.
    Manages Telegram's Flood Control by waiting when necessary.
    Saves sent links to prevent duplicates.

    Args:
        news_items (list): A list of tuples, each containing
                           (title, link, summary, source_name_for_emoji_lookup).
    """
    print("\n--- Starting to send news to Telegram ---")
    if not news_items:
        print("No new items to send.")
        return

    sent_count = 0 # Removed filtered_count as keyword filtering is removed

    for title, link, summary, source_name_for_emoji_lookup in news_items:
        # Skip if the link has already been sent in a previous run.
        if link in sent_links:
            print(f"SKIPPED (already sent): {title}")
            continue

        # Format the message using the dedicated function.
        # This function now expects 4 arguments including source_name_for_emoji_lookup
        formatted_msg = format_telegram_post(title, link, summary, source_name_for_emoji_lookup)

        try:
            # Attempt to send the message.
            await bot.send_message(chat_id=TELEGRAM_CHANNEL_ID, text=formatted_msg, parse_mode=ParseMode.HTML)
            save_sent_link(link) # Save the link after successful sending.
            print(f"SUCCESS: '{title}' sent to Telegram.")
            sent_count += 1
            await asyncio.sleep(3) # Wait to prevent hitting Telegram's rate limits.
        except RetryAfter as e:
            # Handle Telegram's flood control: wait the specified time and retry.
            wait_time = e.retry_after + 1
            print(f"Telegram Flood Control: Waiting for {wait_time} seconds before retrying...")
            await asyncio.sleep(wait_time)

            try:
                # Retry sending the same message.
                await bot.send_message(chat_id=TELEGRAM_CHANNEL_ID, text=formatted_msg, parse_mode=ParseMode.HTML)
                save_sent_link(link)
                print(f"✅SUCCESS (retry): '{title}' sent to Telegram after waiting.")
                sent_count += 1
                await asyncio.sleep(3) # Wait again after successful retry.
            except Exception as retry_e:
                print(f"❌ERROR (retry failed): Could not send '{title}' after waiting: {retry_e}")
        except TimedOut:
            # Handle connection timeouts: wait briefly and continue to the next item.
            print(f"❌TIMEOUT: Telegram API connection timed out for '{title}'. Retrying in 5 seconds.")
            await asyncio.sleep(5)
        except Exception as e:
            # Catch any other unexpected errors during sending.
            print(f"❌ERROR: Could not send '{title}' to Telegram: {e}")

    print(f"--- Finished sending news to Telegram ---")
    print(f"Total items sent: {sent_count}")


In [93]:
import time # Ensure 'import time' is at the very top of your script or Colab notebook.

async def main_bot_run():
    """
    Orchestrates the entire news bot operation:
    1. Loads previously sent links.
    2. Collects news from all configured sources.
    3. Sends new, filtered news items to Telegram.
    4. Measures and prints the total execution time of the run.
    """
    start_time = time.time() # Capture the start time for performance measurement.

    global sent_links # Declare 'sent_links' as global to ensure proper access and modification.
    sent_links = load_sent_links() # Load the set of links that have already been sent.

    print(f"\n--- Bot run started at: {time.ctime()} ---") # Log the start time of the bot's execution.

    collected_news = collect_all_news() # Gather news items from all defined sources (RSS, GitHub, HN).
    await send_news_to_telegram(collected_news) # Process and send the collected news to Telegram.

    end_time = time.time() # Capture the end time after all operations are complete.
    duration = end_time - start_time # Calculate the total duration of the run.

    print(f"--- Bot run finished at: {time.ctime()} ---") # Log the finish time.
    print(f"Total execution time: {duration:.2f} seconds") # Print the total time taken.

# Execute the main asynchronous function.
# This line should be placed at the very end of your script in a Colab environment.
await main_bot_run()


--- Bot run started at: Sun Jul 13 17:53:05 2025 ---

--- Starting News Collection ---
Retrieving news from RSS: Hugging Face Blog (https://huggingface.co/blog/rss.xml)
Retrieving news from RSS: arXiv NLP (http://export.arxiv.org/rss/cs.CL)
Retrieving news from RSS: OpenAI Blog (https://openai.com/blog/rss.xml)
Retrieving news from RSS: Google AI Blog (https://ai.googleblog.com/feeds/posts/default)
Retrieving news from RSS: Papers With Code (https://paperswithcode.com/rss)
Retrieving news from RSS: DeepMind Blog (https://deepmind.com/blog/feed/basic/)
Retrieving news from RSS: Kaggle Blog (https://www.kaggle.com/blog/rss/)
Retrieving news from RSS: Microsoft AI Blog (https://blogs.microsoft.com/ai/feed/)
Retrieving news from RSS: Meta AI Blog (https://ai.meta.com/blog/feed/)
Retrieving news from RSS: Amazon Science Blog (https://www.amazon.science/rss.xml)
Retrieving news from RSS: Towards Data Science (https://medium.com/feed/towards-data-science)
Retrieving news from GitHub Trending

CancelledError: 